In [1]:
import yfinance as yf
import pandas as pd
import time
import random
from utilities import writeOut, gemini, gamma_param,alicloud_llm
from datetime import datetime   
import os

import re
import numpy as np
from google.genai import types
from google import genai

nation_ending={'UK':'.L','US':''}

In [2]:
key=os.getenv("DASHSCOPE_API_KEY")
alicloud_llm('民营银行五年期存款利率排名','你是银行专家',key)

KeyboardInterrupt: 

In [ ]:
list_name='SP500'
annotated_list=pd.read_csv('data/'+list_name+'_annotation_v1.csv',index_col=0)

In [2]:
list_name='FTSE100'
annotated_list=pd.read_csv('data/'+list_name+'_annotation.csv',index_col=0)

In [3]:
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))


In [4]:

def get_price_change(stock,current_price,lookback):
    """
    Calculate percentage change between current price and closing price from x trading days ago
    Args:
        symbol (str): Stock ticker symbol (e.g., 'AAPL')
        lookback_days (int): Number of trading days to look back
    
    Returns:
        float: Percentage change (positive for increase, negative for decrease)
    """
    try:
        
        # Get historical data with buffer for market closures
        hist = stock.history(period='1y',interval='1d')  # Extra buffer
        
        # Get reference price (x trading days ago)
        ref_price = hist['Close'].iloc[-1-lookback]

        
        return ((current_price - ref_price) / ref_price) * 100
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def explain_change(full_name,percentage_change,client):
    #print('activated')
    question='Today is '+datetime.today().strftime('%d-%m-%Y')+'. The stock of company '+full_name+' changed '+str(percentage_change)+' percent. What happened to the company '+full_name+'? Why is there such a dramatic change in its stock price'
    response=gemini(question,client)
    return response

In [5]:

def get_intraday_losers_annotation(annotated_list,nation,sleep_mean=1,sleep_std=2,attribution_threshold=2.0,look_back=[5,20,251],save=True):
    """Get top N intraday losers from a predefined S&P 500 ticker list with company details
    
    Returns DataFrame with company name, sector, market cap, current price, and percentage change.
    Includes random sleep intervals between API calls to prevent rate limiting.
    """
    sleep1,sleep2=gamma_param(sleep_mean,sleep_std)
    ending=nation_ending[nation]
    losers = dict()
    

    for ticker,info in annotated_list.iterrows():
        print(ticker)
        try:
            # Random sleep between 0 and 3 seconds
            #time.sleep(random.uniform(0, 1))
            
            stock = yf.Ticker(ticker+ending)
            data = stock.history(period='1d', interval='1m')
            
            if len(data) < 2:
                continue
                
            latest_price = data['Close'].iloc[-1]
            prev_close = stock.info.get('regularMarketPreviousClose', None)
            
            if prev_close and latest_price:
                pct_change = (latest_price - prev_close) / prev_close * 100
                info['Price']=round(latest_price, 2)
                info['Change_percent']=round(pct_change, 2)
                losers[ticker]=info

                info['Attribution']=''
                print(info['Change_percent'])
                if np.abs(info['Change_percent'])>=attribution_threshold:
                    #print(str(pct_change))
                    reason=explain_change(info['Company Name'],info['Change_percent'],client)
                    print(reason)
                    info['Attribution']=reason
                for k in look_back:
                    info['Change_percent_'+str(k)+'_days']=get_price_change(stock,info['Price'],k)

        except Exception as e:
            print(f"Error {ticker}: {str(e)}")
            info=dict()
        losers[ticker]=info
        sleep_time=np.random.gamma(sleep1,sleep2)
        print(sleep_time)
        time.sleep(sleep_time)
        
    losers_df =pd.DataFrame.from_dict(losers).T
    print(losers_df)
    #print(losers_df.T.columns)
    losers_df = losers_df.sort_values(by='Change_percent')
    if save:
        losers_df.to_csv('daily_ranking/'+list_name+'_top_loser_'+datetime.today().strftime('%Y-%m-%d-%H-%M-%S')+'.csv')
    return losers_df#[['Ticker', 'Company Name', 'Sector', 'Market Cap', 'Price', '% Change']].reset_index(drop=True)


In [6]:
res=get_intraday_losers_annotation(annotated_list,'UK')

III
-0.21


$III.L: possibly delisted; no price data found  (period=1y)
$III.L: possibly delisted; no price data found  (period=1y)


Error: single positional indexer is out-of-bounds
Error: single positional indexer is out-of-bounds


$III.L: possibly delisted; no price data found  (period=1y)


Error: single positional indexer is out-of-bounds
0.050451220411887855
ADM
-0.27
0.8098204441057694
AAF
1.23
0.0036861110222315336
ALW
-0.2
0.022191233276770098
AAL
-0.29
0.1585121856873792
ANTO
0.43
1.195058895639992
AHT
0.02
0.026638426791401226
ABF
0.32
4.969154281987509e-06
AZN
1.77
0.06602174189867382
AUTO
0.18
0.013061879636818678
AV
1.04
0.6444773929123767
BA
0.22
0.4841318145418411
BARC
0.5


...

Error: single positional indexer is out-of-bounds


ERROR:yfinance:$SMIN.L: possibly delisted; no price data found  (period=1y)


Error: single positional indexer is out-of-bounds


ERROR:yfinance:$SMIN.L: possibly delisted; no price data found  (period=1y)


Error: single positional indexer is out-of-bounds
0.14439977219596548
SPX
-0.77
0.00013983692557073082
SSE
-0.46
0.020292916576836472
STJ
-1.37
5.14570753306043
STAN
1.2
0.00116707693729889
TW
-1.83
0.15011621751210907
TSCO
-0.43
0.11467477696194382
ULVR
-0.96
2.587450124220707
UTG
-1.72
0.09360708146996391
UU
-0.57
1.6240260829428774
VOD
0.86
0.13215156846042717
WEIR
-0.54
0.09480525904551246
WTB
-1.73
6.03973391185857e-05
WPP
-0.06
9.636122206554866
                               Company Name                  Sector  \
III                            3i Group Ord                     NaN   
ADM                       Admiral Group plc      Financial Services   
AAF                       Airtel Africa Plc  Communication Services   
ALW                      Alliance Witan Ord                     NaN   
AAL                      Anglo American plc         Basic Materials   
...                                     ...                     ...   
UU               United Utilities Group PLC    

In [ ]:
import smtplib
from email.message import EmailMessage

# Email configuration
sender_email = "deepbelief69@gmail.com"
receiver_email = "wl_xxz@outlook.com"
password = ""  # Use an app-specific password for Gmail

# Create the email message
msg = EmailMessage()
msg["Subject"] = "Hello from Python"
msg["From"] = sender_email
msg["To"] = receiver_email
msg.set_content("This is a plain text email sent via Python.")

# Send the email
try:
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:  # For Gmail (SSL)
        server.login(sender_email, password)
        server.send_message(msg)
    print("Email sent successfully!")
except Exception as e:
    print(f"Error: {e}")

Error: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials ffacd0b85a97d-395c7df3419sm1913480f8f.9 - gsmtp')


In [9]:
import smtplib
from email.message import EmailMessage

# Outlook credentials
sender_email = "wl_xxz@outlook.com"  # or @hotmail.com, @office365.com
password = "Mshfls8139!"  # Use an app password if 2FA is enabled
receiver_email = "xx806@cam.ac.uk"

# Create the email
msg = EmailMessage()
msg["Subject"] = "Test from Outlook via Python"
msg["From"] = sender_email
msg["To"] = receiver_email
msg.set_content("This email was sent using Outlook's SMTP server!")

try:
    # Connect to Outlook's SMTP server
    with smtplib.SMTP("smtp.office365.com", 587) as server:
        server.starttls()  # Enable encryption
        server.login(sender_email, password)
        server.send_message(msg)
    print("Email sent successfully!")
except Exception as e:
    print(f"Failed to send email: {e}")

Failed to send email: (535, b'5.7.139 Authentication unsuccessful, basic authentication is disabled. [LO4P123CA0384.GBRP123.PROD.OUTLOOK.COM 2025-03-13T12:17:39.251Z 08DD61EA3B382F0F]')


In [28]:
res=get_intraday_losers_annotation(annotated_list,'US')

AAPL
-0.48
Error: single positional indexer is out-of-bounds
3.351853942069543
NVDA
2.82
On February 18, 2025, NVIDIA Corporation's stock experienced a change of approximately -2.63%. Several factors could be contributing to this change:

*   **Billionaire Investor Activity:** Israel Englander, CEO of hedge fund Millennium Management, reduced his position in NVIDIA by selling 1.1 million shares in Q4 2024. Philippe Laffont also sold 80% of Coatue's stake in NVIDIA.
*   **AI Infrastructure Spending:** Major tech companies like Amazon, Microsoft, and Meta are planning massive investments in AI infrastructure, totaling up to $320 billion for 2025. This is generally seen as a positive sign for NVIDIA, as they dominate the AI accelerator market.
*   **Upcoming Earnings Report:** NVIDIA is scheduled to report its Q4 earnings on February 26th. Expectations are high, and any deviation from these expectations could impact the stock price. Wall Street anticipates revenue of $38 billion and earni

ERROR:yfinance:$HWM: possibly delisted; no price data found  (period=1y)


Error: single positional indexer is out-of-bounds


ERROR:yfinance:$HWM: possibly delisted; no price data found  (period=1y)


Error: single positional indexer is out-of-bounds


ERROR:yfinance:$HWM: possibly delisted; no price data found  (period=1y)


Error: single positional indexer is out-of-bounds
2.7222478911522954
DHI
-0.11
Error: single positional indexer is out-of-bounds
4.157171514153722
KR
0.35
Error: single positional indexer is out-of-bounds
1.5501442369302638
NDAQ
0.23
Error: single positional indexer is out-of-bounds
3.799022252367914
OXY
0.67
Error: single positional indexer is out-of-bounds
0.25817470814070087
EW
0.41
Error: single positional indexer is out-of-bounds
0.7596097474682725
COR
-0.51
Error: single positional indexer is out-of-bounds
1.9250914984842862
KDP
0.43
Error: single positional indexer is out-of-bounds
5.91984137338785
FIS
1.06
Error: single positional indexer is out-of-bounds
2.1790952097038843
KMB
0.86
Error: single positional indexer is out-of-bounds
1.9544703766408595
NEM
0.73
Error: single positional indexer is out-of-bounds
1.502259489453131
DFS
1.17
Error: single positional indexer is out-of-bounds
4.242211669652906
PCG
-0.16
Error: single positional indexer is out-of-bounds
2.996430328889391

ERROR:yfinance:$INCY: possibly delisted; no price data found  (period=1d)


BXP
Error BXP: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
0.7496343543217079
KMX
-0.6
Error: single positional indexer is out-of-bounds
3.155917931427796
CAG
-7.73


KeyboardInterrupt: 

In [16]:
list_name='ftse100'
ending='.L'
# list_name='SP500.txt'
# ending=''
filel=open('../gathering/data/'+list_name+'.txt','a+')
filel.seek(0)
stocklist=filel.read().splitlines()
filel.close()

In [18]:
res.to_csv(list_name+'_top_loser_'+datetime.today().strftime('%Y-%m-%d')+'.csv', index=False)


In [61]:
question="Please summarise the business of UK stock symbol GSK. Return one sentence within 500 characters."

In [64]:
question='deepseek api how to enable web search? what input to use'

In [11]:
question='what date is it today? what time is it now'#'what happened to GSK stock today?'
#question='Who is Matthias Zilbauer'

In [48]:
question='If i am to create an annotation file for a list of stocks, what can be useful information? I can think of market cap, beta, a summary of its business, place of registration'

Today is Wednesday, February 5, 2025, and the time is 8:00:49 PM UTC.



In [49]:

messages = [
    {'role': 'system', 'content': 'You are a financial analyst'}, #Python programmer
    {'role': 'user', 'content': question}
    ]
response = Generation.call(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key = "sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"), 
    model="deepseek-r1-distill-llama-70b",   # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=messages,
    result_format="message"
)

if response.status_code == 200:
    print(response.output.choices[0].message.content)
else:
    print(f"HTTP返回码：{response.status_code}")
    print(f"错误码：{response.code}")
    print(f"错误信息：{response.message}")
    print("请参考文档：https://help.aliyun.com/zh/model-studio/developer-reference/error-code")

<think>
Okay, so the user wants to create an annotation file for a list of stocks and is asking what useful information to include. They've already thought of market cap, beta, a business summary, and place of registration. That's a solid start, but I can probably help them think of more comprehensive details.

First, I should consider who the user is. They mentioned they're a financial analyst, so they likely need detailed and actionable data. Their main goal is to create an annotation file, which I assume will be used for analysis, reporting, or maybe feeding into a model.

The user's initial ideas cover some fundamentals: market cap gives size, beta shows volatility, business summary provides context, and place of registration indicates jurisdiction. But maybe they need more granular information depending on their use case. For instance, if they're doing a valuation, financial metrics like P/E ratio or dividend yield would be important. If they're assessing risk, including beta is g

In [51]:

re.sub('<think>[^>]+</think>\n\n', '', response.output.choices[0].message.content)

"Creating a comprehensive annotation file for a list of stocks is a valuable task for analysis, reporting, or even machine learning applications. Below is a list of **useful information** you can include in your annotation file, depending on your goals and the level of detail required:\n\n---\n\n### **1. Fundamental Information**\n- **Ticker Symbol**: The stock ticker (e.g., AAPL for Apple Inc.).\n- **Company Name**: The full name of the company.\n- **Industry**: The industry the company operates in (e.g., Technology, Healthcare, Financials).\n- **Sector**: The broader sector (e.g., Information Technology, Consumer Discretionary).\n- **Key Products/Services**: A brief summary of the company's main offerings.\n- **Revenue Streams**: Primary sources of revenue (e.g., hardware sales, software subscriptions, advertising).\n- **Geographic Presence**: Where the company operates (e.g., global, regional, specific countries).\n\n---\n\n### **2. Financial Metrics**\n- **Market Capitalization (Ma

'<think>\nOkay, so the user wants to write a Python program that removes content between the words " Includes" and "Please". They provided an example where the input is a string with those words, and the output should exclude the text in between, keeping the rest.\n\nFirst, I need to figure out how to identify the start and end points. The problem mentions " Includes" and "Please", so I should look for those exact strings. But wait, the input example shows " Includes" with a capital \'I\' and a space before it, while "Please" is lowercase. I need to make sure the code accounts for case sensitivity. But in the example, the code seems to be case-sensitive. Maybe the user wants an exact match.\n\nNext, I should split the string into parts. The idea is to split the string into three parts: before " Includes", the part to be removed, and after "Please". So, using split with a maxsplit of 2 would give me three parts.\n\nWait, in Python, the split method can take a maxsplit parameter. So, if 

In [ ]:

def get_intraday_losers(tickers, ending=''):
    """Get top N intraday losers from a predefined S&P 500 ticker list with company details
    
    Returns DataFrame with company name, sector, market cap, current price, and percentage change.
    Includes random sleep intervals between API calls to prevent rate limiting.
    """
    losers = []
    
    def get_market_cap_formatted(market_cap):
        """Convert market cap to readable format"""
        if not market_cap:
            return 'N/A'
        billion = 1_000_000_000
        if market_cap >= billion:
            return f"${round(market_cap/billion, 1)}B"
        else:
            return f"${round(market_cap/1_000_000, 1)}M"
    
    for ticker in tickers:
        print(ticker)
        try:
            # Random sleep between 0 and 3 seconds
            time.sleep(random.uniform(0, 1))
            
            stock = yf.Ticker(ticker+ending)
            data = stock.history(period='1d', interval='1m')
            
            if len(data) < 2:
                continue
                
            latest_price = data['Close'].iloc[-1]
            prev_close = stock.info.get('regularMarketPreviousClose', None)
            
            if prev_close and latest_price:
                pct_change = (latest_price - prev_close) / prev_close * 100
                losers.append({
                    'Ticker': ticker,
                    'Company Name': stock.info.get('longName', 'N/A'),
                    'Sector': stock.info.get('sector', 'N/A'),
                    'Market Cap': get_market_cap_formatted(stock.info.get('marketCap', None)),
                    'Price': round(latest_price, 2),
                    '% Change': round(pct_change, 2)
                })
        except Exception as e:
            continue
    
    losers_df = pd.DataFrame(losers)
    if len(losers_df) == 0:
        return pd.DataFrame(columns=['Ticker', 'Company Name', 'Sector', 'Market Cap', 'Price', '% Change'])
    
    losers_df = losers_df.sort_values(by='% Change')
    return losers_df[['Ticker', 'Company Name', 'Sector', 'Market Cap', 'Price', '% Change']].reset_index(drop=True)
